In [1]:
%pip install tpot
%pip install torch
%pip install pip-utils

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [33]:
import sys
import pandas as pd
import os
import time
from datetime import datetime
import numpy as np
import multiprocessing as mp
import torch
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from tpot import TPOTClassifier



In [63]:
data_list = []
for i in range(7):
    data = pd.read_csv('FIFA-Data/Cleaned Data/new_players_' + str(15 + i) + '.csv').set_index('sofifa_id')
    data_list.append(data)

In [64]:
ndata_list = []
for i in range(6):
    next_yr_value = data_list[i+1]['value_eur']
    #ndata_list.append(data_list[i].join(next_yr_value, how='left', rsuffix='_next').fillna(0))
    ndata_list.append(data_list[i])
    ndata_list[i]['value_eur_next'].fillna(0)

In [65]:
# Loading Outfielder Data
ndata_list2 = []
for i, j in enumerate(ndata_list):
    ndata_list2.append(j.where(j['value_eur_next']!=0.0).dropna(how='all'))
    ndata_list2[i] = ndata_list2[i].where(ndata_list2[i]['team_position']!='GK').dropna(how='all')

tier1 = ['English Premier League','Spain Primera Division','Italian Serie A',
        'German 1. Bundesliga', 'French Ligue 1', 'Russian Premier League',
        'Portuguese Liga ZON SAGRES', 'Belgian Jupiler Pro League',
        'Holland Eredivisie', 'USA Major League Soccer']

tier2 = ['Turkish Süper Lig','Mexican Liga MX','Campeonato Brasileiro Série A',
        'Scottish Premiership', 'Argentina Primera División',
        'Austrian Football Bundesliga', 'Ukrainian Premier League',
        'Danish Superliga', 'Swiss Super League', 'Rest of World']

tier3 = ['Croatian Prva HNL','English League Championship', 'Greek Super League', 
        'Swedish Allsvenskan', 'Czech Republic Gambrinus Liga',
        'Saudi Abdul L. Jameel League', 'Polish T-Mobile Ekstraklasa',
        'Chinese Super League', 'Colombian Liga Postobón',
        'Japanese J. League Division 1']

tier4 = ['Norwegian Eliteserien','Romanian Liga I','0',
        'German 2. Bundesliga','Spanish Segunda División', 
        'Chilian Campeonato Nacional','Australian Hyundai A-League', 
        'Italian Serie B', 'French Ligue 2',
        'South African Premier Division', 
        'English League One', 'Korean K League Classic',
        'German 3. Bundesliga', 'English League Two',
        'Finnish Veikkausliiga', 'Rep. Ireland Airtricity League']

rankinggk = [[],[],[],[],[],[]]
for j in range(6):
    for i in ndata_list2[j]['league_name']:
        if i in tier1:
            rankinggk[j].append(1)
        elif i in tier2:
            rankinggk[j].append(2)
        elif i in tier3:
            rankinggk[j].append(3)
        elif i in tier4:
            rankinggk[j].append(4)
        else:
            rankinggk[j].append(5)

for i in range(len(ndata_list2)):
    ndata_list2[i].insert(10,'league_ranking',rankinggk[i])
    
for i in range(len(ndata_list2)):
    ndata_list2[i].insert(8,'year',np.ones(len(ndata_list2[i]))*i)

In [66]:
# Below is the set of columns we care about

features = ['age'
            ,'height_cm'
           ,'weight_kg'
           ,'overall'
           ,'potential'
           ,'pace'
           ,'shooting'
           ,'passing'
           ,'defending'
           ,'dribbling'
           ,'physic'
           ,'year'
           ,'league_ranking']

filtered_data = []
ys = []

for d in ndata_list2:
    filtered_data.append(d[features].fillna(0))
    ys.append(d['value_eur_next'].fillna(0))

# Merging data across years
for i in range(len(filtered_data)):
    n = len(filtered_data[i])
    t1 = int(0.65*n)
    t2 = int(0.8*n)
    index_shuffle = np.array(filtered_data[i].index)
    np.random.shuffle(index_shuffle)

    train_x_i = filtered_data[i].loc[index_shuffle[0:t1]]
    validate_x_i = filtered_data[i].loc[index_shuffle[t1:t2]]
    test_x_i = filtered_data[i].loc[index_shuffle[t2:n]]

    train_y_i = ys[i].loc[index_shuffle[0:t1]]
    validate_y_i = ys[i].loc[index_shuffle[t1:t2]]
    test_y_i = ys[i].loc[index_shuffle[t2:n]]
    
    train_log_y_i = np.log(train_y_i)
    validate_log_y_i = np.log(validate_y_i)
    test_log_y_i = np.log(test_y_i)
    train_log_y_i.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
    validate_log_y_i.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
    test_log_y_i.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
    if i == 0:
        train_x = train_x_i
        validate_x = validate_x_i
        test_x = test_x_i
        train_y = train_y_i
        validate_y = validate_y_i
        test_y = test_y_i        
        train_log_y = train_log_y_i
        validate_log_y = validate_log_y_i
        test_log_y = test_log_y_i  
    else:
        train_x = train_x.append(train_x_i)
        validate_x = validate_x.append(validate_x_i)
        test_x = test_x.append(test_x_i)
        train_y = train_y.append(train_y_i)
        validate_y = validate_y.append(validate_y_i)
        test_y = test_y.append(test_y_i)        
        train_log_y = train_log_y.append(train_log_y_i)
        validate_log_y = validate_log_y.append(validate_log_y_i)
        test_log_y = test_log_y.append(test_log_y_i)
    model = LinearRegression()
    model.fit(train_x,train_y)
    pred = model.coef_.reshape(13,1)
train_x

,age,height_cm,weight_kg,overall,potential,pace,shooting,passing,defending,dribbling,physic,year,league_ranking
sofifa_id,,,,,,,,,,,,,
179712,25.0,175.0,67.0,71.0,75.0,78.0,67.0,68.0,32.0,76.0,55.0,0.0,1
185204,28.0,192.0,84.0,69.0,69.0,53.0,59.0,63.0,71.0,51.0,75.0,0.0,2
202423,25.0,180.0,79.0,56.0,62.0,60.0,45.0,51.0,29.0,58.0,58.0,0.0,1
192310,25.0,184.0,77.0,68.0,71.0,78.0,58.0,61.0,66.0,66.0,73.0,0.0,1
190483,23.0,170.0,73.0,80.0,84.0,85.0,73.0,78.0,51.0,85.0,62.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239728,24.0,183.0,73.0,64.0,66.0,68.0,62.0,59.0,40.0,64.0,69.0,5.0,3
232593,25.0,185.0,74.0,69.0,72.0,77.0,68.0,51.0,18.0,65.0,58.0,5.0,3
229294,28.0,193.0,89.0,70.0,70.0,50.0,30.0,45.0,70.0,41.0,79.0,5.0,4


In [44]:
from tpot import TPOTRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# TPOT Regressor - Outfielders

tpot1 = TPOTRegressor(generations=5, population_size=20, verbosity=2, max_time_mins=1.5)
tpot1.fit(train_x, train_y)
pred = tpot1.predict(test_x)
print("Test Set Error: ", str(np.mean(np.abs(test_y - np.array(pred).reshape(len(pred),)))))

Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


1.68 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=False, max_features=0.5, min_samples_leaf=7, min_samples_split=2, n_estimators=100)
Test Set Error:  982764.793111172


In [68]:
# TPOT Classifier - Outfielders

tpot2 = TPOTClassifier(generations=5, population_size=20, verbosity=2, max_time_mins=1.5)
tpot2.fit(train_x, train_y)
pred = tpot2.predict(test_x)
print("Test Set Error: ", str(np.mean(np.abs(test_y - np.array(pred).reshape(len(pred),)))))

Test Set Error:  1131130.159857904


In [55]:
# TPOT Classifier - Outfielders with bins

bins_try= [[0,600000,2500000,150000000],[0,500000,1250000,5000000,150000000],[0,400000,900000,2500000,10000000,150000000],
           [0,300000,700000,1250000,3000000,8000000,150000000],[0,250000,600000,1000000,2000000,4000000,10000000,150000000],
          [0,200000,500000,800000,1250000,2250000,4500000,10000000,150000000]]

import copy
ys_bins1 = copy.deepcopy(ys) #3
ys_bins2 = copy.deepcopy(ys) #4
ys_bins3 = copy.deepcopy(ys) #5
ys_bins4 = copy.deepcopy(ys) #6
ys_bins5 = copy.deepcopy(ys) #7
ys_bins6 = copy.deepcopy(ys) #8
ys_bins = [ys_bins1,ys_bins2,ys_bins3,ys_bins4,ys_bins5,ys_bins6]
for h in range(len(ys_bins)):
    for i in range(len(ys_bins[h])):
        for j in ys_bins[h][i].index:
            for k, l in enumerate(bins_try[h]):
                if l >= ys_bins[h][i][j]:
                    ys_bins[h][i][j] = k
                    break
                    
train_y = [[],[],[],[],[],[]]
validate_y = [[],[],[],[],[],[]]
test_y = [[],[],[],[],[],[]]
for i in range(len(filtered_data)):
    n = len(filtered_data[i])
    t1 = int(0.65*n)
    t2 = int(0.8*n)
    index_shuffle = np.array(filtered_data[i].index)
    np.random.shuffle(index_shuffle)

    train_x_i = filtered_data[i].loc[index_shuffle[0:t1]]
    validate_x_i = filtered_data[i].loc[index_shuffle[t1:t2]]
    test_x_i = filtered_data[i].loc[index_shuffle[t2:n]]
    if i == 0:
        train_x = train_x_i
        validate_x = validate_x_i
        test_x = test_x_i
    else:
        train_x = train_x.append(train_x_i)
        validate_x = validate_x.append(validate_x_i)
        test_x = test_x.append(test_x_i)
    for j in range(len(ys_bins)):
        train_y_i = ys_bins[j][i].loc[index_shuffle[0:t1]]
        validate_y_i = ys_bins[j][i].loc[index_shuffle[t1:t2]]
        test_y_i = ys_bins[j][i].loc[index_shuffle[t2:n]]
        if i == 0:
            train_y[j] = train_y_i
            validate_y[j] = validate_y_i
            test_y[j] = test_y_i
        else:
            train_y[j] = train_y[j].append(train_y_i)
            validate_y[j] = validate_y[j].append(validate_y_i)
            test_y[j] = test_y[j].append(test_y_i)

preds_c = []
for h in range(len(ys_bins)):
    tpot_c1 = TPOTClassifier(generations=5, population_size=20, verbosity=2, max_time_mins=1.5)
    tpot_c1.fit(train_x, train_y[h])
    preds_c.append((h, tpot_c1.predict(test_x)))



Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7600291019149059

Generation 2 - Current best internal CV score: 0.7634025958908096

1.50 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.4, min_samples_leaf=4, min_samples_split=14, n_estimators=100)


Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7035096909376637

Generation 2 - Current best internal CV score: 0.7035096909376637

1.54 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: GaussianNB(XGBClassifier(input_matrix, learning_rate=0.1, max_depth=10, min_child_weight=12, n_estimators=100, n_jobs=1, subsample=0.45, verbosity=0))


Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]

...


Generation 1 - Current best internal CV score: 0.574122577265584

1.54 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(ZeroCount(input_matrix), bootstrap=False, criterion=entropy, max_features=0.8500000000000001, min_samples_leaf=18, min_samples_split=8, n_estimators=100)


Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.5514184273325184

1.53 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.7000000000000001, min_samples_leaf=16, min_samples_split=2, n_estimators=100)


Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


1.51 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.9500000000000001, min_samples_leaf=9, min_samples_split=3, n_estimators=100)
Test Set Error:  0 	 1.185907436297452
Test Set Error:  1 	 0.9550182007280291
Test Set Error:  2 	 0.8619344773790951
Test Set Error:  3 	 0.9175767030681228
Test Set Error:  4 	 1.015340613624545
Test Set Error:  5 	 1.388715548621945


In [60]:
for h in range(len(preds_c)):
    print("Test Set Error: ", str(preds_c[h][0]),"\t", str(np.sum(preds_c[h][1]==test_y[h])/len(test_y[h])))

Test Set Error:  0 	 0.8065522620904836
Test Set Error:  1 	 0.750390015600624
Test Set Error:  2 	 0.6786271450858035
Test Set Error:  3 	 0.5741029641185648
Test Set Error:  4 	 0.5647425897035881
Test Set Error:  5 	 0.5195007800312013


In [73]:
# Loading Goalkeeper only data
ndata_list3 = []
for i, j in enumerate(ndata_list):
    ndata_list3.append(j.where(j['value_eur_next']!=0.0).dropna(how='all'))
    ndata_list3[i] = ndata_list3[i].where(ndata_list3[i]['team_position']=='GK').dropna(how='all')

rankinggk = [[],[],[],[],[],[]]
for j in range(6):
    for i in ndata_list3[j]['league_name']:
        if i in tier1:
            rankinggk[j].append(1)
        elif i in tier2:
            rankinggk[j].append(2)
        elif i in tier3:
            rankinggk[j].append(3)
        elif i in tier4:
            rankinggk[j].append(4)
        else:
            rankinggk[j].append(5)

for i in range(len(ndata_list3)):
    ndata_list3[i].insert(10,'league_ranking',rankinggk[i])
    
for i in range(len(ndata_list3)):
    ndata_list3[i].insert(8,'year',np.ones(len(ndata_list3[i]))*i)

features = ['age'
            ,'height_cm'
           ,'weight_kg'
           ,'overall'
           ,'potential'
           ,'gk_diving'
           ,'gk_reflexes'
           ,'gk_handling'
           ,'gk_kicking'
           ,'gk_speed'
           ,'gk_positioning'
           ,'year'
           ,'league_ranking']

filtered_data = []
ys = []
for d in ndata_list3:
    filtered_data.append(d[features].fillna(0))
    ys.append(d['value_eur_next'].fillna(0))



In [ ]:
# TPOT Regressor - Goalkeepers

tpot3 = TPOTRegressor(generations=5, population_size=20, verbosity=2, max_time_mins=1.5)
tpot3.fit(train_x, train_y)
pred = tpot3.predict(test_x)
print("Test Set Error: ", str(np.mean(np.abs(test_y - np.array(pred).reshape(len(pred),)))))

In [76]:
# TPOT Classifier - Goalkeepers

tpot3 = TPOTClassifier(generations=5, population_size=20, verbosity=2, max_time_mins=1.5)
tpot3.fit(train_x, train_y)
pred = tpot3.predict(test_x)
print("Test Set Error: ", str(np.mean(np.abs(test_y - np.array(pred).reshape(len(pred),)))))

Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


1.83 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.8500000000000001, min_samples_leaf=7, min_samples_split=8, n_estimators=100)
Test Set Error:  1020633.5346358792


In [62]:
# TPOT Classifier - Goalkeepers with bins

bins_try= [[0,600000,2500000,150000000],[0,500000,1250000,5000000,150000000],[0,400000,900000,2500000,10000000,150000000],
           [0,300000,700000,1250000,3000000,8000000,150000000],[0,250000,600000,1000000,2000000,4000000,10000000,150000000],
          [0,200000,500000,800000,1250000,2250000,4500000,10000000,150000000]]

import copy
ys_bins1 = copy.deepcopy(ys) #3
ys_bins2 = copy.deepcopy(ys) #4
ys_bins3 = copy.deepcopy(ys) #5
ys_bins4 = copy.deepcopy(ys) #6
ys_bins5 = copy.deepcopy(ys) #7
ys_bins6 = copy.deepcopy(ys) #8
ys_bins = [ys_bins1,ys_bins2,ys_bins3,ys_bins4,ys_bins5,ys_bins6]
for h in range(len(ys_bins)):
    for i in range(len(ys_bins[h])):
        for j in ys_bins[h][i].index:
            for k, l in enumerate(bins_try[h]):
                if l >= ys_bins[h][i][j]:
                    ys_bins[h][i][j] = k
                    break
                    
train_y = [[],[],[],[],[],[]]
validate_y = [[],[],[],[],[],[]]
test_y = [[],[],[],[],[],[]]
for i in range(len(filtered_data)):
    n = len(filtered_data[i])
    t1 = int(0.65*n)
    t2 = int(0.8*n)
    index_shuffle = np.array(filtered_data[i].index)
    np.random.shuffle(index_shuffle)

    train_x_i = filtered_data[i].loc[index_shuffle[0:t1]]
    validate_x_i = filtered_data[i].loc[index_shuffle[t1:t2]]
    test_x_i = filtered_data[i].loc[index_shuffle[t2:n]]
    if i == 0:
        train_x = train_x_i
        validate_x = validate_x_i
        test_x = test_x_i
    else:
        train_x = train_x.append(train_x_i)
        validate_x = validate_x.append(validate_x_i)
        test_x = test_x.append(test_x_i)
    for j in range(len(ys_bins)):
        train_y_i = ys_bins[j][i].loc[index_shuffle[0:t1]]
        validate_y_i = ys_bins[j][i].loc[index_shuffle[t1:t2]]
        test_y_i = ys_bins[j][i].loc[index_shuffle[t2:n]]
        if i == 0:
            train_y[j] = train_y_i
            validate_y[j] = validate_y_i
            test_y[j] = test_y_i
        else:
            train_y[j] = train_y[j].append(train_y_i)
            validate_y[j] = validate_y[j].append(validate_y_i)
            test_y[j] = test_y[j].append(test_y_i)

preds_c = []
for h in range(len(ys_bins)):
    tpot_c2 = TPOTClassifier(generations=5, population_size=20, verbosity=2, max_time_mins=1.5)
    tpot_c2.fit(train_x, train_y[h])
    preds_c.append((h, tpot_c2.predict(test_x)))

for h in range(len(preds_c)):
    print("Test Set Error: ", str(preds_c[h][0]),"\t", str(np.sum(preds_c[h][1]==test_y[h])/len(test_y[h])))

Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


1.54 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: GradientBoostingClassifier(PolynomialFeatures(input_matrix, degree=2, include_bias=False, interaction_only=False), learning_rate=0.01, max_depth=7, max_features=0.05, min_samples_leaf=18, min_samples_split=2, n_estimators=100, subsample=1.0)


Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.6962877597345905

1.51 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.55, min_samples_leaf=6, min_samples_split=5, n_estimators=100)


Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]

...


1.53 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=False, criterion=entropy, max_features=0.55, min_samples_leaf=1, min_samples_split=16, n_estimators=100)


Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


1.54 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=0.55, min_samples_leaf=15, min_samples_split=6, n_estimators=100)


Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


1.62 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.7000000000000001, min_samples_leaf=2, min_samples_split=18, n_estimators=100)
Test Set Error:  0 	 0.7800312012480499
Test Set Error:  1 	 0.7145085803432137
Test Set Error:  2 	 0.6458658346333853
Test Set Error:  3 	 0.5881435257410297
Test Set Error:  4 	 0.5756630265210608
Test Set Error:  5 	 0.516380655226209
